In [1]:
%run "01_link a bolt and a nut on a axes.ipynb"

In [2]:
def ops3():
    
    axes=ops2()
    
    return axes
    

In [3]:
if not "__file__" in locals():
    import asyncio
    await asyncio.sleep(0)

In [4]:
if not "__file__" in locals():
    axes=ops3()
    assert compare_data(*get_bolt_and_nut(axes),"bf_Pitch") is True

In [5]:
axes

bpy.data.objects['Empty']

In [6]:
bolt,nut=get_bolt_and_nut(axes)

In [7]:
bolt.data.keys()

<IDPropertyGroupViewKeys "RNA_struct_idprops">

In [8]:
def screwing(bolt,nut):
    def to_screw(bolt,nut):
        pitch=bolt.data["bf_Pitch"]

        constraint=bolt.constraints.new(type='TRANSFORM')

        constraint.map_from = 'LOCATION'

        constraint.target_space = 'LOCAL'
        constraint.owner_space = 'LOCAL'
        constraint.map_to = 'ROTATION'


        constraint.to_min_z_rot = 6.28319
        constraint.to_max_z_rot = 0
        constraint.from_max_z = pitch
        constraint.from_min_z = 0

        constraint.target = nut

        constraint.use_motion_extrapolate = True
    to_screw(bolt,nut)
    to_screw(nut,bolt)

In [9]:
screwing(bolt,nut)

In [10]:
def limiting(nut,bolt):
    length=bolt.data["bf_Thread_Length"]
    length-=nut.data['bf_Hex_Nut_Height']
    def to_limit_location(nut,negative=False):
        nonlocal length
        
        constraint=nut.constraints.new(type="LIMIT_LOCATION")

        constraint.owner_space = 'LOCAL'
        
        constraint.use_min_x = True
        constraint.use_max_x = True
        constraint.use_min_y = True
        constraint.use_max_y = True
        
        constraint.use_min_z = True
        constraint.use_max_z = not negative
        
        constraint.min_x=0
        constraint.max_x=0
        constraint.min_y=0
        constraint.max_y=0
        
        if negative:
            constraint.min_z=-length
            constraint.max_z=0
        else:
            constraint.min_z=0
            constraint.max_z=length
        
        constraint.use_transform_limit = True

        
    def to_limit_rotation(nut):
        constraint=nut.constraints.new(type="LIMIT_ROTATION")

        constraint.owner_space = 'LOCAL'
        
        constraint.use_limit_x = True
        constraint.use_limit_y = True
        constraint.use_limit_z = False
        
        constraint.min_x=0
        constraint.min_y=0
        constraint.min_z=0
        constraint.max_x=0
        constraint.max_y=0
        constraint.max_z=0
        
        
        constraint.use_transform_limit = True

    to_limit_location(nut)
    to_limit_location(bolt,True)
    
    to_limit_rotation(nut)
    to_limit_rotation(bolt)
    

In [11]:
limiting(nut,bolt)